# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Scale the humidity values to control marker size
city_data_df['MarkerSize'] = city_data_df['Humidity'] / 10  # Adjust scaling if necessary


map_plot = city_data_df.hvplot.scatter(
    x='Lng', y='Lat',  
    size='MarkerSize',  # Size based on humidity
    color='Humidity',   
    hover_cols=['City', 'Humidity'],  
    title='City Humidity Map',
    xlabel='Longitude', ylabel='Latitude',
    size_max=30,  
    cmap='Blues',  
)

# Display the map
map_plot


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit specific criteria (example criteria)
# For instance, filtering cities with humidity > 50% and temperature > 20°C
city_data_df = city_data_df[(city_data_df['Humidity'] > 50) & (city_data_df['Max Temp'] < 40)]

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display a sample of the data
city_data_df.head()  


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Step 1: Create a copy of the relevant columns
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Step 2: Add an empty column "Hotel Name"
hotel_df['Hotel Name'] = None

# Step 3: Display a sample of the data
hotel_df.head()


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
import requests

# Set parameters to search for a hotel
radius = 10000  # Search radius of 10 km
params = {
    'apiKey': geoapify_key,  # Use your Geoapify API key stored in geoapify_key
    'radius': radius,  
    'categories': 'accommodation.hotel',  # Specify for hotels
}

# Print a message to follow up on the hotel search
print("Starting hotel search")

# Iterate through the dataframe.
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']  
    lon = row['Lng']  

    # Add the current city's latitude and longitude to the params dictionary
    params["bias"] = f"proximity:{lon},{lat}"  

    # Print the full URL for debugging purposes
    url = f"https://api.geoapify.com/v2/places?{requests.compat.urlencode(params)}"
    print(f"Requesting URL: {url}")

    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  
        
        # Print the raw response for further inspection
        print(f"Response Status Code: {response.status_code}")
        print(f"Response Content: {response.text}")

        name_address = response.json()

        # Log the response structure for debugging
        if 'features' in name_address and len(name_address['features']) > 0:
            hotel_info = name_address["features"][0]["properties"]
            # Check if the "name" key exists in the properties
            if "name" in hotel_info:
                hotel_df.loc[index, "Hotel Name"] = hotel_info["name"]
            else:
                hotel_df.loc[index, "Hotel Name"] = "Hotel name not available"
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    except requests.exceptions.RequestException as e:
        # Handle request exceptions (network issues, invalid API, etc.)
        print(f"Error with request for {hotel_df.loc[index, 'City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "Request failed"

# Display sample data
hotel_df


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Create a map plot with the hotel coordinates (Lat, Lng)
hotel_map = hotel_df.hvplot.points(
    x='Lng',  # Longitude
    y='Lat',  # Latitude
    geo=True,  # Use geo projection to plot on a map
    tiles='OSM',  # Use OpenStreetMap tiles for map background
    size=10,  # Marker size 
    hover_cols=['City', 'Hotel Name'],  # Display City and Hotel Name when hovered
)



In [ ]:
hotel_map